In [2]:
#importing modudles
import pandas as pd
import numpy as np
import time
import math
import recordlinkage

In [3]:
dfA = pd.read_csv('CleanedInputDB.csv')
dfA['SSN'] = dfA['SSN'].astype(str)

In [20]:
start_time = time.time()
# Indexation step
pcl = recordlinkage.SortedNeighbourhoodIndex('LastName', window=3)
pairs = pcl.index(dfA)
# Comparison step
compare_cl = recordlinkage.Compare()
compare_cl.string('LastName', 'LastName', method='jaro', threshold=0.75, label='LastName')
compare_cl.string('FirstName', 'FirstName', method='jaro', threshold=0.75, label='FirstName')
compare_cl.string('SSN', 'SSN', method='jaro', threshold=0.75, label='SSN')

features = compare_cl.compute(pairs, dfA)
end_time = time.time()
total_time = end_time - start_time

In [21]:
print(total_time)

24.81622004508972


In [22]:
# Classification step
matches = features[features.sum(axis=1) > 2]

In [23]:
headers=['tuple_id_1', 'tuple_id_2']
print(len(matches))
res =[]

72437


In [24]:
for i in range(len(matches)):
    name0 = matches.iloc[i].name[0]
    name1 = matches.iloc[i].name[1]
    first = dfA.iloc[name0]['RecID']
    second = dfA.iloc[name1]['RecID']
    res.append([first,second])
    if(i%10000 == 0):
        print('Passed', i)

Passed 0
Passed 10000
Passed 20000
Passed 30000
Passed 40000
Passed 50000
Passed 60000
Passed 70000


In [25]:
dfout = pd.DataFrame(res, columns=headers)

In [26]:
dfout.to_csv('OutputDB-Task2.csv', index=False)

In [27]:
bashCommand = "python web_client-Task2.py"
import subprocess
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
print(output)

b'Duplicate Detection Results:\nPrecision = 0.95679003824\nRecall = 0.280339771463\nF1 = 0.433626768274\n'
